In [17]:
import torch
from hydra.utils import instantiate
from hydra import initialize, compose
import hydra

from data.dataManager import DataManager
from model.modelCreator import ModelCreator
from omegaconf import OmegaConf
from scripts.run import run

In [18]:
hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(version_base=None, config_path="config")

hydra.initialize()

In [19]:
config=compose(config_name="config.yaml")

In [20]:
model, dataMgr = run(config)

[17:37:27.064] INFO   data.dataManager                                  Keys: ['incident_energies', 'showers']
[17:37:30.160] INFO   data.dataManager                                  dict_keys(['incident_energies', 'showers'])
[17:37:30.163] INFO   data.dataManager                                  <torch.utils.data.dataloader.DataLoader object at 0x7f7543139d90>: 101816 events, 796 batches
[17:37:30.164] INFO   data.dataManager                                  <torch.utils.data.dataloader.DataLoader object at 0x7f75430966f0>: 12728 events, 13 batches
[17:37:30.164] INFO   data.dataManager                                  <torch.utils.data.dataloader.DataLoader object at 0x7f74a1efd970>: 12727 events, 13 batches
[17:37:30.165] INFO   model.modelCreator                                ::Creating Model
[17:37:30.239] INFO   scripts.run                                       Requesting GPUs. GPU list :[2]
[17:37:30.241] INFO   scripts.run                                       Main GPU : cuda

encoder._networks.0.seq1.0.conv.weight True
encoder._networks.0.seq1.0.conv.bias True
encoder._networks.0.seq1.1.weight True
encoder._networks.0.seq1.1.bias True
encoder._networks.0.seq1.2.weight True
encoder._networks.0.seq1.3.conv.weight True
encoder._networks.0.seq1.3.conv.bias True
encoder._networks.0.seq1.4.weight True
encoder._networks.0.seq1.4.bias True
encoder._networks.0.seq1.5.weight True
encoder._networks.0.seq2.0.conv.weight True
encoder._networks.0.seq2.0.conv.bias True
encoder._networks.0.seq2.1.weight True
encoder._networks.0.seq2.1.bias True
encoder._networks.0.seq2.2.weight True
encoder._networks.0.seq2.3.conv.weight True
encoder._networks.0.seq2.3.conv.bias True
encoder._networks.0.seq2.4.weight True
encoder._networks.1.seq1.0.conv.weight True
encoder._networks.1.seq1.0.conv.bias True
encoder._networks.1.seq1.1.weight True
encoder._networks.1.seq1.1.bias True
encoder._networks.1.seq1.2.weight True
encoder._networks.1.seq1.3.conv.weight True
encoder._networks.1.seq1.3.

[17:37:31.468] INFO   scripts.run                                       Model NOT being watched by wandb


In [ ]:
# model.decoder

AttributeError: 'AutoEncoderBase' object has no attribute 'decoder'

In [4]:
dataMgr = DataManager(config)

#create model handling object
modelCreator = ModelCreator(config)

#instantiate the chosen model
#loads from file 
model=modelCreator.init_model()
model.create_networks()

[21:29:23.165] INFO   data.dataManager                                  Keys: ['incident_energies', 'showers']
[21:29:26.236] INFO   data.dataManager                                  dict_keys(['incident_energies', 'showers'])
[21:29:26.239] INFO   data.dataManager                                  <torch.utils.data.dataloader.DataLoader object at 0x7f125d56ce60>: 101816 events, 796 batches
[21:29:26.240] INFO   data.dataManager                                  <torch.utils.data.dataloader.DataLoader object at 0x7f1448425dc0>: 12728 events, 13 batches
[21:29:26.241] INFO   data.dataManager                                  <torch.utils.data.dataloader.DataLoader object at 0x7f125fc6d730>: 12727 events, 13 batches


In [6]:
model.print_model_info()

In [10]:
model.__dict__.items()
for key,par in model.__dict__.items():
    if isinstance(par,torch.Tensor):
        print("{0}: {1}".format(key, par.shape))
    else:
        print("{0}: {1}".format(key, par))

training: True
_parameters: {}
_buffers: {}
_non_persistent_buffers_set: set()
_backward_pre_hooks: OrderedDict()
_backward_hooks: OrderedDict()
_is_full_backward_hook: None
_forward_hooks: OrderedDict()
_forward_hooks_with_kwargs: OrderedDict()
_forward_hooks_always_called: OrderedDict()
_forward_pre_hooks: OrderedDict()
_forward_pre_hooks_with_kwargs: OrderedDict()
_state_dict_hooks: OrderedDict()
_state_dict_pre_hooks: OrderedDict()
_load_state_dict_pre_hooks: OrderedDict()
_load_state_dict_post_hooks: OrderedDict()
_modules: {'encoder': HierarchicalEncoder(
  (smoothing_dist_mod): GumbelMod(
    (activation_fct): Sigmoid()
  )
  (_networks): ModuleList(
    (0-3): 4 x EncoderBlockPBH3Dv3Reg(
      (seq1): Sequential(
        (0): PeriodicConv3d(
          (conv): Conv3d(1, 32, kernel_size=(1, 3, 3), stride=(1, 1, 2))
        )
        (1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): PReLU(num_parameters=32)
        (3): PeriodicConv3

In [7]:
from model.encoder.encoderhierarchybase import HierarchicalEncoder

In [8]:
enc = HierarchicalEncoder(config)

In [13]:
x = next(iter(dataMgr.train_loader))
enc(x[0].to(dtype=torch.float32), x[1].to(dtype=torch.float32))

(tensor(5.),
 [tensor([[ 5.3311e-02, -1.1581e-01,  7.8747e-02,  ...,  1.5792e-04,
           -4.9996e-02,  3.5318e-02],
          [ 6.2303e-02, -9.8599e-02,  7.3160e-02,  ..., -2.6149e-02,
           -4.2177e-02,  3.7049e-02],
          [ 5.2927e-02, -1.1597e-01,  8.6535e-02,  ..., -1.1240e-02,
           -4.6329e-02,  3.9148e-02],
          ...,
          [ 5.7003e-02, -1.0246e-01,  7.5525e-02,  ..., -1.6580e-02,
           -3.7898e-02,  4.6485e-02],
          [ 5.2346e-02, -1.1706e-01,  7.9303e-02,  ..., -4.8886e-03,
           -3.7729e-02,  4.3794e-02],
          [-2.3839e-02, -1.2354e-01,  2.1129e-01,  ...,  1.2532e-01,
            2.4063e-02,  1.0864e-02]], grad_fn=<ClampBackward1>),
  tensor([[ 0.0192, -0.1110,  0.0105,  ...,  0.0808,  0.0155,  0.1094],
          [ 0.0214, -0.0860,  0.0300,  ...,  0.0967,  0.0448,  0.0861],
          [ 0.0203, -0.0941,  0.0091,  ...,  0.0892,  0.0418,  0.0920],
          ...,
          [ 0.0299, -0.1018,  0.0114,  ...,  0.0936,  0.0355,  0.0776],